# Notebook 2 - SQL avec vraies bases de données
## Analyse e-commerce avec PostgreSQL en ligne




### 🎯 Objectifs pédagogiques
- Connecter Python à une vraie base de données PostgreSQL
- Écrire des requêtes SQL complexes sur des données réelles
- Implémenter des analyses RFM avec SQL
- Intégrer SQL et pandas pour des analyses avancées
- Gérer les connexions et la sécurité

### 🛍️ Contexte du projet
Vous analysez les données d'un vrai dataset e-commerce (Brazilian E-Commerce Public Dataset) hébergé sur une base PostgreSQL.

Objectif : créer une segmentation clientèle pour optimiser les campagnes marketing.


## Partie 1 : Connexion à la base de données réelle

### 🔧 Installation et configuration


# Installation des dépendances


```
pip install psycopg2-binary sqlalchemy pandas python-dotenv
```




In [1]:
pip install sqlalchemy psycopg2-binary python-dotenv pandas

Note: you may need to restart the kernel to use updated packages.


In [21]:
import psycopg2 # Permet de se connecter à une base de données PostgreSQL directement en Python
import pandas as pd # Bibliothèque pour manipuler des données sous forme de tableaux (DataFrame)
import numpy as np  # Fournit des outils pour les calculs numériques (matrices, statistiques, etc.)
from sqlalchemy import create_engine # Crée un "moteur" pour se connecter aux bases de données (PostgreSQL, MySQL, etc.)
import matplotlib.pyplot as plt  # faire des graphiques et des visualisations
import seaborn as sns # Bibliothèque basée sur matplotlib pour faire de beaux graphiques statistiques
from datetime import datetime, timedelta # Permet de gérer et manipuler des dates et durées ajouter des jours...
import os   # Sert à interagir avec le système d'exploitation exp : lire des variables d'environnement
from dotenv import load_dotenv # Permet de charger les variables stockées dans un fichier .env : MDP, clés API...

### 🌐 Base de données PostgreSQL gratuite (ElephantSQL)

**Option 1 : ElephantSQL (20MB gratuit)**
1. Créez un compte sur [elephantsql.com](https://www.elephantsql.com/)
2. Créez une instance "Tiny Turtle" (gratuite)
3. Récupérez vos credentials

**Option 2 : Supabase (500MB gratuit)**
1. Créez un compte sur [supabase.com](https://supabase.com/)
2. Créez un nouveau projet
3. Récupérez l'URL de connexion PostgreSQL

In [22]:
# Charger les variables d'environnement depuis le fichier .env pour eviter de mettre les mdp dans le code 
load_dotenv()

# Créer un dictionnaire contenant les informations de connexion à la base de données
DATABASE_CONFIG = {
    'user': os.getenv('DB_USER'), # Nom d'utilisateur pour la base 
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),# Adresse du serveur
    'port': os.getenv('DB_PORT'),# Port utilisé pour PostgreSQL
    'database': os.getenv('DB_NAME'), 
}

# Création de l'engine SQLAlchemy
# Créer le moteur SQLAlchemy avec les informations du dictionnaire DATABASE_CONFIG
engine = create_engine(
    f"postgresql://{DATABASE_CONFIG['user']}:{DATABASE_CONFIG['password']}@"
     #postgresql : indique le type de base de données (dialecte), on utilise PostgreSQL.
    f"{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"
)

In [23]:

# Test de connexion
def test_connection(): # Définir une fonction pour tester la connexion à la base de données
    """
    Testez votre connexion à la base

    Étapes :
    1. Utilisez pd.read_sql() pour exécuter "SELECT version()"
    2. Affichez la version PostgreSQL
    3. Gérez les erreurs de connexion
    """
    try:
        # Exécute une requête SQL pour récupérer la version de PostgreSQL
        # pd.read_sql() retourne un DataFrame (tableau Pandas)
        version = pd.read_sql("SELECT version();", engine)
        print("Connexion réussie :")
        print(version)
    except Exception as e:
        print(f"Erreur de connexion : {e}")
        return False
    return True


# Exécuter le test
test_connection()


Connexion réussie :
                                             version
0  PostgreSQL 17.4 on aarch64-unknown-linux-gnu, ...


True


## Partie 2 : Import du dataset e-commerce

### 📊 Dataset Brazilian E-Commerce
Nous utilisons le célèbre dataset Olist (100k commandes réelles).

**Tables à créer :**
1. **customers** : customer_id, customer_city, customer_state
2. **orders** : order_id, customer_id, order_status, order_date, order_delivered_date
3. **order_items** : order_id, product_id, seller_id, price, freight_value
4. **products** : product_id, product_category, product_weight_g
5. **sellers** : seller_id, seller_city, seller_state

In [24]:
# 1°Chargement des fichiers Olist
customers = pd.read_csv("data/olist_customers_dataset.csv")
orders = pd.read_csv("data/olist_orders_dataset.csv")
order_items = pd.read_csv("data/olist_order_items_dataset.csv")
products = pd.read_csv("data/olist_products_dataset.csv")
sellers = pd.read_csv("data/olist_sellers_dataset.csv")

In [ ]:
#2°CREATION TAB
def create_tables(engine):
    from sqlalchemy import text

    create_customers = text("""
    CREATE TABLE IF NOT EXISTS customers (
        customer_id VARCHAR PRIMARY KEY,
        customer_city VARCHAR(100),
        customer_state CHAR(2)
    );
    """)

    create_sellers = text("""
    CREATE TABLE IF NOT EXISTS sellers (
        seller_id VARCHAR PRIMARY KEY,
        seller_city VARCHAR(100),
        seller_state CHAR(2)
    );
    """)

    create_products = text("""
    CREATE TABLE IF NOT EXISTS products (
        product_id VARCHAR PRIMARY KEY,
        product_category VARCHAR,
        product_weight_g NUMERIC
    );
    """)

    create_orders = text("""
    CREATE TABLE IF NOT EXISTS orders (
        order_id VARCHAR PRIMARY KEY,
        customer_id VARCHAR,
        order_status VARCHAR(50),
        order_date TIMESTAMP,
        order_delivered_date TIMESTAMP,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );
    """)

    create_order_items = text("""
    CREATE TABLE IF NOT EXISTS order_items (
        order_id VARCHAR,
        product_id VARCHAR,
        seller_id VARCHAR,
        price NUMERIC,
        freight_value NUMERIC,
        FOREIGN KEY (order_id) REFERENCES orders(order_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id),
        FOREIGN KEY (seller_id) REFERENCES sellers(seller_id)
    );
    """)

    with engine.connect() as conn:
        conn.execute(create_customers)
        conn.execute(create_sellers)
        conn.execute(create_products)
        conn.execute(create_orders)
        conn.execute(create_order_items)
        conn.commit()

In [26]:
#3°
create_tables(engine)

# Insertion des données dans Supabase :


In [ ]:
#Insertion des données dans Supabase dans un seule bloc 
try:
    # 1. Insertion des données dans la table 'customers'
    customers_filtered = customers[["customer_id", "customer_city", "customer_state"]]
    customers_filtered.to_sql("customers", engine, if_exists="append", index=False)
    print("Données insérées dans 'customers'")

    # 2. Insertion des données dans la table 'sellers'
    sellers_filtered = sellers[["seller_id", "seller_city", "seller_state"]]
    sellers_filtered.to_sql("sellers", engine, if_exists="append", index=False)
    print("Données insérées dans 'sellers'")

    # 3. Insertion des données dans la table 'products'
    products_filtered = products[["product_id", "product_weight_g"]]
    products_filtered.to_sql("products", engine, if_exists="append", index=False)
    print("Données insérées dans 'products'")

    # 4. Insertion des données dans la table 'orders'
    orders_renamed = orders.rename(columns={
        "order_purchase_timestamp": "order_date",
        "order_delivered_customer_date": "order_delivered_date"
    })
    orders_filtered = orders_renamed[[
        "order_id", "customer_id", "order_status", "order_date", "order_delivered_date"
    ]]
    orders_filtered.to_sql("orders", engine, if_exists="append", index=False)
    print("Données insérées dans 'orders'")

    # 5. Insertion des données dans la table 'order_items' (sans doublons sur la clé primaire composite)
    order_items_filtered = order_items[[
        "order_id", "product_id", "seller_id", "price", "freight_value"
    ]].drop_duplicates(subset=["order_id", "product_id", "seller_id"])
    order_items_filtered.to_sql("order_items", engine, if_exists="append", index=False)
    print("Données insérées dans 'order_items' sans doublons.")

except Exception as e:
    print("Une erreur s’est produite lors de l’insertion des données :", e)


Données insérées dans 'customers'
Données insérées dans 'sellers'
Données insérées dans 'products'
Données insérées dans 'orders'
Données insérées dans 'order_items' sans doublons.


In [ ]:
#9°
print(sellers.shape)
print(products.shape)
print(orders.shape)
print(order_items.shape)

(3095, 4)
(32951, 9)
(99441, 8)
(112650, 7)


## Partie 3 : Requêtes SQL avancées


### 🔍 Analyses SQL à implémenter

#### 1. Analyse RFM (Récence, Fréquence, Montant)
```sql
-- Votre défi : Calculer les métriques RFM pour chaque client
WITH customer_metrics AS (
    SELECT
        c.customer_id,
        c.customer_state,
        -- Récence : jours depuis dernier achat
        -- Fréquence : nombre de commandes
        -- Montant : total dépensé
        
        -- Complétez cette requête CTE
        
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_id, c.customer_state
)

-- Créez les segments RFM (Champions, Loyaux, À risque, etc.)
SELECT
    customer_id,
    customer_state,
    recency_score,
    frequency_score,
    monetary_score,
    CASE
        WHEN recency_score >= 4 AND frequency_score >= 4 THEN 'Champions'
        WHEN recency_score >= 3 AND frequency_score >= 3 THEN 'Loyal Customers'
        -- Ajoutez les autres segments
        ELSE 'Others'
    END as customer_segment
FROM customer_metrics;
```

In [ ]:
#Requête SQL de base : lire une table
from sqlalchemy import text

query_customers = """
SELECT * FROM customers
LIMIT 10
"""

with engine.begin() as connection:
    df_customers = pd.read_sql(query_customers, con=connection)


In [ ]:
#### 2. Analyse géographique des ventes

def geographic_sales_analysis():
    """
    Analysez les performances par état/région

    Requêtes à écrire :
    1. Top 10 des états par CA
    2. Croissance MoM par région
    3. Taux de conversion par ville
    4. Distance moyenne vendeur-acheteur
    """

    query_top_states = """
    -- Votre requête SQL ici
    -- Utilisez des JOINs et GROUP BY
    -- Calculez le CA, nombre de commandes, panier moyen
    """

    return pd.read_sql(query_top_states, engine)

#### 3. Analyse temporelle et saisonnalité
```sql
-- Détectez les patterns saisonniers
SELECT
    EXTRACT(YEAR FROM order_date) as year,
    EXTRACT(MONTH FROM order_date) as month,
    EXTRACT(DOW FROM order_date) as day_of_week,
    COUNT(*) as order_count,
    SUM(price + freight_value) as total_revenue,
    AVG(price + freight_value) as avg_order_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE order_status = 'delivered'
GROUP BY ROLLUP(
    EXTRACT(YEAR FROM order_date),
    EXTRACT(MONTH FROM order_date),
    EXTRACT(DOW FROM order_date)
)
ORDER BY year, month, day_of_week;
```

---

## Partie 4 : Analyse prédictive avec SQL

### 🔮 Modèles simples en SQL

In [ ]:
#### 1. Prédiction de churn

def churn_prediction_sql():
    """
    Identifiez les clients à risque de churn

    Indicateurs :
    - Pas d'achat depuis X jours
    - Baisse de fréquence d'achat
    - Diminution du panier moyen
    - Changement de comportement géographique
    """

    churn_query = """
    WITH customer_activity AS (
        -- Calculez les métriques d'activité récente
        -- Comparez avec l'historique du client
        -- Scorez le risque de churn
    )

    SELECT
        customer_id,
        days_since_last_order,
        order_frequency_trend,
        monetary_trend,
        churn_risk_score,
        CASE
            WHEN churn_risk_score > 0.7 THEN 'High Risk'
            WHEN churn_risk_score > 0.4 THEN 'Medium Risk'
            ELSE 'Low Risk'
        END as churn_segment
    FROM customer_activity;
    """

    return pd.read_sql(churn_query, engine)


#### 2. Recommandations produits
```sql
-- Market Basket Analysis simplifié
WITH product_pairs AS (
    SELECT
        oi1.product_id as product_a,
        oi2.product_id as product_b,
        COUNT(*) as co_purchase_count
    FROM order_items oi1
    JOIN order_items oi2 ON oi1.order_id = oi2.order_id
    WHERE oi1.product_id != oi2.product_id
    GROUP BY oi1.product_id, oi2.product_id
    HAVING COUNT(*) >= 10  -- Seuil minimum
)

SELECT
    product_a,
    product_b,
    co_purchase_count,
    co_purchase_count::float / total_a.count as confidence
FROM product_pairs pp
JOIN (
    SELECT product_id, COUNT(*) as count
    FROM order_items
    GROUP BY product_id
) total_a ON pp.product_a = total_a.product_id
ORDER BY confidence DESC;
```

---

## Partie 5 : Intégration avec les APIs météo

### 🌤️ Croisement données météo/ventes
```python
def weather_sales_correlation():
    """
    Correlez vos données météo du Notebook 1 avec les ventes
    
    Hypothèses à tester :
    1. Les ventes de certaines catégories augmentent-elles avec la pluie ?
    2. Y a-t-il un impact de la température sur les achats ?
    3. Les livraisons sont-elles impactées par la météo ?
    """
    
    # Récupérez les données météo historiques pour les villes brésiliennes
    weather_query = """
    SELECT DISTINCT customer_city, customer_state
    FROM customers
    WHERE customer_state IN ('SP', 'RJ', 'MG', 'RS', 'SC')
    ORDER BY customer_city;
    """
    
    cities = pd.read_sql(weather_query, engine)
    
    # Intégrez avec l'API météo
    # Analysez les corrélations
    
    pass
```

### 📊 Dashboard géo-temporel
```python
def create_geotemporal_dashboard():
    """
    Créez un dashboard interactif combinant :
    - Carte des ventes par région
    - Évolution temporelle avec météo
    - Segments clients géolocalisés
    - Prédictions par zone géographique
    """
    pass
```

---
## 🏆 Livrables finaux

### 📈 Rapport d'analyse complet
1. **Segmentation RFM (Recency, Frenquency, Monetary) ** : 5-7 segments avec caractéristiques
2. **Analyse géographique**  : Performances par région + recommandations
3. **Prédictions churn** : Liste des clients à risque + actions
4. **Recommandations produits** : Top 10 des associations
5. **Impact météo** : Corrélations significatives identifiées

### 🚀 Pipeline automatisé
```python
def automated_analysis_pipeline():
    """
    Pipeline qui :
    1. Se connecte à la DB
    2. Exécute toutes les analyses
    3. Met à jour les segments clients
    4. Génère le rapport automatiquement
    5. Envoie des alertes si nécessaire
    """
    pass
```

---

## 🎓 Auto-évaluation

- [ ] **Connexion DB** : PostgreSQL fonctionnelle
- [ ] **Requêtes complexes** : JOINs, CTEs, fonctions analytiques
- [ ] **Gestion des erreurs** : Connexions robustes
- [ ] **Performance** : Requêtes optimisées avec index
- [ ] **Intégration** : SQL + Python + APIs
- [ ] **Insights actionables** : Recommandations business claires

### 🔗 Préparation au Notebook 3
Le prochain notebook portera sur NoSQL (MongoDB) avec des données de réseaux sociaux et d'IoT, en temps réel.

### 💡 Bases de données alternatives
- **PlanetScale** : MySQL serverless gratuit
- **MongoDB Atlas** : 512MB gratuit
- **FaunaDB** : Base multi-modèle gratuite
- **Hasura Cloud** : GraphQL + PostgreSQL